In [23]:
import os, sys,re,json
from pathlib import Path
import fitz,pdfplumber
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()
from langchain_core.documents import Document
from typing import Optional,Dict,List
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os,sys
from pathlib import Path
# Base_dir = Path(__file__).resolve().parent.parent
# sys.path.append(str(Base_dir))
# from Bot.Data_preparation import extract_text_from_pdf,merge_number_line,detect_heading,detect_tables_text
from pathlib import Path
import pdfplumber
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_classic.memory import ConversationBufferWindowMemory
from langchain_google_genai import GoogleGenerativeAI,ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_astradb import AstraDBVectorStore
from langchain_classic.retrievers import EnsembleRetriever,ContextualCompressionRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document
from langchain_classic.retrievers.document_compressors import LLMChainExtractor,LLMChainFilter,DocumentCompressorPipeline
from langchain_classic.chains import RetrievalQA
from langchain_community.document_transformers import LongContextReorder
from langchain_community.document_compressors import FlashrankRerank
from langchain_core.documents import Document as LCDocument
from langchain_core.messages import AIMessage, HumanMessage
import os,sys
from pathlib import Path 
# Base_dir = Path(__file__).resolve().parent.parent
# sys.path.append(str(Base_dir))
from Bots.Data_preparation import attach_images_to_paragraphs,extract_text,detect_images,pdf_paths
from langchain_core.documents import Document
from langchain_astradb import AstraDBVectorStore
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_classic.retrievers import ContextualCompressionRetriever,EnsembleRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor,LLMChainFilter,DocumentCompressorPipeline
from langchain_community.document_transformers import LongContextReorder
from langchain_community.document_compressors import FlashrankRerank
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_groq.chat_models import ChatGroq
from typing import Sequence,Annotated,List,Optional,Dict,Literal
from langchain_classic import hub
from langgraph.graph import MessagesState,StateGraph,END,START
from langgraph.prebuilt import ToolNode,tools_condition
from pydantic import BaseModel,Field
from typing_extensions import TypedDict
from langchain.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langchain_classic.agents import Tool,create_react_agent,create_self_ask_with_search_agent
from langchain_community.tools import TavilySearchResults,TavilyAnswer
load_dotenv()
from langchain_classic.prompts import ChatPromptTemplate,PromptTemplate
from langchain_core.output_parsers import StrOutputParser,PydanticOutputParser

doc_path = Path.cwd() / "Data"
pdf_paths = doc_path / "ISO-7001-2023.pdf"

#Detect Images:
def detect_images(pdf_path, output_dir="images"):
    os.makedirs(output_dir, exist_ok=True)
    doc = fitz.open(pdf_path)
    images = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap()
        image_name = f"page_{page_num+1}.png"
        image_path = os.path.join(output_dir, image_name)
        pix.save(image_path)

        images.append({
            "page_num": page_num + 1,
            "image_path": image_path
        })
        # blocks = page.get_text("dict")["blocks"]

        # img_index = 0
        # for block in blocks:
        #     if block["type"] == 1 and "xref" in block:  # image block
        #         bbox = block["bbox"]
        #         pix = page.get_pixmap(clip = fitz.Rect(bbox))
        #         if pix.n > 4:
        #             pix = fitz.Pixmap(fitz.csRGB, pix)

        #         image_name = f"page_{page_num+1}_Img_{img_index}.png"
        #         image_path = os.path.join(output_dir, image_name)
        #         pix.save(image_path)

        #         images.append({
        #             "page_num": page_num + 1,
        #             "bbox": bbox,
        #             "image_path": image_path
        #         })

        #         img_index += 1
                #   pix = None
    doc.close()
    return images
detect_images(pdf_paths)


#Detect Headings:
def detect_heading(line:str,font_info:Optional[Dict]=None)->str:
    line = line.strip()
    if not line:
        return None
    
    match = re.match(r'^(\d+(?:\.\d+){0,3})\s+(.+)$', line)
    if match:
        prefix = match.group(1)
        dot_count = prefix.count(".")

        if dot_count ==0:
           classification = "heading"
        elif dot_count==1:
            classification = "sub_heading"
        elif dot_count ==2:
            classification = "sub_sub_heading"
        else:
            classification = None

        if font_info:
            font_name = font_info.get("font","").strip.lower()
            font_size = font_info.get("size",0)
            if "bold" or "italic" in font_info or font_size>=12:
                return classification
            else:
                return None
        
        return classification
        
    #Non numbered but bold
    if font_info:
        font_name = font_info.get("font", "").lower()
        font_size = font_info.get("size", 0)
        if "bold" in font_name and font_size >= 10:
            return "sub_heading"
        
    #Number headings
    only_number = re.match(r'^(\d+(?:\.\d+){0,3})$', line)
    if only_number:
        prefix = only_number.group(1)
        dot_count = prefix.count(".")

        if dot_count == 0:
            return "heading"
        elif dot_count == 1:
            return "sub_heading"
        
    return None

#Extraction of text:
def extract_text(pdf_path):
    doc =fitz.open(pdf_path)
    structured_data = []
    current_section = None
    current_sub_section = None
    current_sub_sub_section = None

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        page_dict = page.get_text("dict")
        page_lines = []
        
        for blocks in page_dict.get("blocks",[]):
            for lines in blocks.get("lines",[]):
                line_text = "\n".join([span.get('text') for span in lines.get("spans",[])])
                # print(repr(line_text))
                if not line_text:
                    continue
                
                if (
                        re.match(r"^ISO\s+\d{4,5}:\d{4}\(E\)$", line_text)
                        or re.search(r"© ISO \d{4} – All rights reserved", line_text)
                   
                    ):
                        continue
                page_lines.append(line_text)     
        # print(page_lines)

        #Paragraph initiation
        current_para = ""
        for line in page_lines:
            line = line.replace('\ufeff', '').replace('\x08', '').replace('\xa0','').strip()
            line = re.sub(r'\s+', ' ', line).strip()
            if not line:
                if current_para:
                    structured_data.append({
                        "Document_name": pdf_path,
                                "page_num": page_num + 1,
                                "heading": current_section,
                                "sub_heading": current_sub_section,
                                "sub_sub_heading": current_sub_sub_section,
                                "paragraph": current_para.strip()
                    })

                    current_para = ""
                continue

            #Detect heading
            classification = detect_heading(line)
            if classification in ["heading","sub_heading","sub_sub_heading"]:
                if current_para:
                    structured_data.append({
                        "Document_name": pdf_path,
                                "page_num": page_num + 1,
                                "heading": current_section,
                                "sub_heading": current_sub_section,
                                "sub_sub_heading": current_sub_sub_section,
                                "paragraph": current_para.strip()
                    })

                    current_para = ""

            # Update trackers
                if classification == "heading":
                    current_section = line
                    current_sub_section = None
                    current_sub_sub_section = None
                elif classification == "sub_heading":
                    current_sub_section = line
                    current_sub_sub_section = None
                elif classification == "sub_sub_heading":
                    current_sub_sub_section = line
            

            if re.match(r"^([a-z]\)|\d+\.|\-|•)\s+.+", line):
                if current_para:
                    current_para += " " + line
                else:
                    current_para = line
            else:
                if current_para:
                    current_para += " " + line
                else:
                    current_para = line

            # Finalize last paragraph on page
        if current_para:
            structured_data.append({
                "Document_name":pdf_path,
                "page_num": page_num + 1,
                "heading": current_section,
                "sub_heading": current_sub_section,
                "sub_sub_heading": current_sub_sub_section,
                "paragraph": current_para.strip()
            })
    doc.close()
    return {"Paragraphs":structured_data}

def attach_images_to_paragraphs(structured_data, images):
    """
    Attach images to all paragraphs under the same heading context
    until the next heading appears.
    """
    # Group paragraphs by page
    page_paragraphs = {}
    for idx, para in enumerate(structured_data):
        page_paragraphs.setdefault(para["page_num"], []).append(idx)

    for img in images:
        pg_num = img["page_num"]
        if pg_num not in page_paragraphs:
            continue

        # Attach image to all paragraphs on the page (context until next heading)
        for idx in page_paragraphs[pg_num]:
            if "images" not in structured_data[idx]:
                structured_data[idx]["images"] = []
            structured_data[idx]["images"].append(img["image_path"])

    return structured_data

pdf_path = pdf_paths
output_dir = "images"

# 1️⃣ Extract text
text_data = extract_text(pdf_path)
paragraphs = text_data["Paragraphs"]

# 2️⃣ Extract images
images = detect_images(pdf_path, output_dir=output_dir)

# 3️⃣ Attach images contextually
final_data = attach_images_to_paragraphs(paragraphs, images)

# 4️⃣ Check sample output
# for item in final_data:
#     print(item)


In [ ]:
ASTRA_DB_API_ENDPOINT=os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN=os.getenv("ASTRA_DB_APPLICATION_TOKEN")
AstraDB_Keyspace = 'default_keyspace'

GOOGLE_API_KEY  = os.getenv("GOOGLE_API_KEY")
Tavily_API_Key = os.getenv("TAVILY_API_KEY") or "tavily_xxx"
Langchain_API_Key = os.getenv("LANGCHAIN_API_KEY") or "lsmith_xxx"
GROQ_API_KEY= os.getenv("GROQ_API_KEY") or "groq_xxx"
# Set environment variables
os.environ["TAVILY_API_KEY"] = Tavily_API_Key
os.environ["LANGCHAIN_API_KEY"] = Langchain_API_Key
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en-v1.5",encode_kwargs={"normalize_embeddings": True})
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    api_key=GROQ_API_KEY,
    timeout=None,
    max_retries=2,
)

def sanitize_metadata(metadata):
    clean_meta = {}

    for k, v in metadata.items():
        key = k.replace(" ", "_").lower()   
        if isinstance(v, Path):
            clean_meta[key] = str(v)
        elif isinstance(v, list):
            clean_meta[key] = [
                str(i) if isinstance(i, Path) else i
                for i in v
            ]
        elif isinstance(v, (str, int, float, bool)) or v is None:
            clean_meta[key] = v
        else:
            clean_meta[key] = str(v)

    return clean_meta

# 1. Extract text
doc = extract_text(pdf_paths)
structured_data = doc["Paragraphs"]

# 2. Extract images
images = detect_images(pdf_paths)

# 3. Attach images
structured_data = attach_images_to_paragraphs(structured_data, images)


def chunk_text(structured_data,chunk_size=550,chunk_overlap=300):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap,separators=["\n\n", "\n", ".", " ", ""])
    text_chunks = []

    for record in structured_data:
        metadata = {
            "Document_name":record.get("Document_name"),
            "Page num":record.get("page_num"),
            "Heading":record.get("heading"),
            "Sub Heading":record.get("sub_heading"),
            "Sub_sub_heading":record.get("sub_sub_heading"),
            "snippet":record.get("images"),
        }

        for chunk in text_splitter.split_text(record.get("paragraph")):
            text_chunks.append({"text":chunk,"metadata":metadata})
    return text_chunks

chunks = chunk_text(structured_data)
all_chunks=[]
for i, chunk in enumerate(chunks):
    # print(i, type(chunk.get("metadata")), chunk.get("metadata"))
    # if not isinstance(chunk.get("metadata"), dict):
    #     print("INVALID METADATA FOUND AT INDEX:", i)
    #     break
    clean_metadata = sanitize_metadata(chunk['metadata'])
    all_chunks.append(Document(page_content=chunk['text'],metadata=clean_metadata))

#2.Embedding
sample_query = "Definition post-market surveillance"
sample_embed = embeddings.embed_query(sample_query)
# print(f"First five dimensions of embeddings were:{sample_embed[:5]}")

#3. Vector store
vector_store = AstraDBVectorStore(
    embedding=embeddings,
    collection_name="Bot_collection_HIL",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token = ASTRA_DB_APPLICATION_TOKEN,
    namespace=AstraDB_Keyspace
)

#4. Injestion:
# vector_store.add_documents(all_chunks)
# print(f"Total chunks created were {len(all_chunks)}")
# print(f" All documents ingested successfully {len(all_chunks)}")

# #5. Retriever
reorder = LongContextReorder()
FlashrankRerank.model_rebuild()
reranker = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2", top_n=10)
# compressor = LLMChainExtractor.from_llm(llm)
# filter = LLMChainFilter.from_llm(llm)
keyword_retriever = BM25Retriever.from_documents(all_chunks)
keyword_retriever.k=5
vector_retriever = vector_store.as_retriever(search_type="similarity",kwargs={"k":5})
hybrid_retriever= EnsembleRetriever(retrievers=[keyword_retriever,vector_retriever],weights=[0.5,0.5])
pipeline = DocumentCompressorPipeline(transformers=[reranker])
retriever = ContextualCompressionRetriever(base_compressor=pipeline,base_retriever=hybrid_retriever)

# if __name__=="__main__":
#     query = "What is Negation?"
#     answer =retriever.invoke(query)
#     print(answer)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
INFO:langchain_astradb.vectorstores:vector store default init, collection 'Bot_collection_HIL'
INFO:root:Detecting API environment 'prod' from supplied endpoint
INFO:astrapy.data.database:createCollection('Bot_collection_HIL')
INFO:httpx:HTTP Request: POST https://da560a44-8a47-47c1-9200-eb2f9e31c431-us-east1.apps.astra.datastax.com/api/json/v1/default_keyspace "HTTP/1.1 200 OK"
INFO:astrapy.data.database:finished createCollection('Bot_collection_HIL')


In [25]:
#Memory:
memory = ConversationBufferWindowMemory(k=17, memory_key="Chat_History",return_messages=True)
def format_chat_history():
    """Read memory and format as string for prompt"""
    chat_history_str = ""
    for message in memory.chat_memory.messages:
        if isinstance(message,HumanMessage):
            chat_history_str +=f"Human message:{message.content}\n"
        elif isinstance(message,AIMessage):
            chat_history_str +=f"Human message:{message.content}\n"
    return chat_history_str.strip()

In [33]:
#Corrective RAG--->Agentic AI

def make_msgpack_safe(obj):
    if isinstance(obj, dict):
        return {k: make_msgpack_safe(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [make_msgpack_safe(v) for v in obj]
    elif isinstance(obj, np.generic):  # numpy.float32, int64, etc.
        return obj.item()
    elif isinstance(obj, (float, int, str, bool)) or obj is None:
        return obj
    else:
        return str(obj)

#Web Prompt:
web_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert assistant.
Use web search results to answer the user's question.
Do NOT say 'I don't know'.
You can also respond to general greetings like "hi", "hello", "who are you", etc. Ignore spelling mistakes.
If spelling mistakes from user understand the intent what they were trying to ask. If people asking some tricky psycological question about your confidence, handle diplomatically.
     
     """),
    ("human", """Conversation so far:
{chat_history}

Current Question:
{question}

Web Results:
{context}
""")
])

#1. Grader node
class grade_docs(BaseModel):
    binary_score: str=Field("Grade the retrieved documents with its relevance and say yes or no")
llm_with_structured_output = llm.with_structured_output(grade_docs)

system = """You are a relevance grader for Corrective RAG.

Rules:
- Compare the document and the user question.
- If the document is relevant, output "Yes".
- If the document is irrelevant, empty, or off-topic, output "No".
- Output ONLY "Yes" or "No". """

prompt = ChatPromptTemplate([
    ("system", system),
    ("human", """Conversation so far:
{chat_history}

Retrieved Document:
{documents}

User Question:
{question}
""")
])
grader = prompt | llm_with_structured_output
# question = "question"
# docs = retriever.invoke(question)
# if not docs or len(docs)==0:
#     result={
#         "binary_score":"No",
#         "reason": "No relevant documents found"
#     }
#     # print(result)
# else:
#     context = "\n".join(d.page_content for d in docs[:3] if d.page_content)

#     if not context.strip():
#         result = { "binary_score":"No",
#         "reason": "Documents are not that releavant"}

#     else:
#         grade_answer = grader.invoke({"question":question,"documents":docs[1].page_content})

    # print(grade_answer)

#2. Generate node
rag_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an assistant for question-answering tasks.

Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

Conversation history is provided for context only.
Do NOT use conversation history as a source of truth unless supported by retrieved context.

You can also respond to general greetings like "hi", "hello", "who are you", etc. Ignore spelling mistakes.
If spelling mistakes from user understand the intent what they were trying to ask. If people asking some tricky psycological question about your confidence, handle diplomatically.

"""
        ),
        (
            "human",
            """Conversation History:
{chat_history}

Retrieved Context:
{context}

Question:
{question}
"""
        )
    ]
)

output_parser = StrOutputParser()


def format_documents(documents):
    formatted = []

    for i, d in enumerate(documents[:3], 1):
        meta = d.get("metadata", {})

        formatted.append(
            f"""[Document {i}]
Source: {meta.get("document_name", "Unknown")}
Page: {meta.get("page_num", "N/A")}
Heading: {meta.get("heading", "N/A")}
Sub Heading: {meta.get("sub_heading", "N/A")}
Sub-sub Heading: {meta.get("sub_sub_heading", "N/A")}
Images: {meta.get("snippet", "N/A")}

Content:
{d["page_content"]}
"""
        )

    return "\n\n".join(formatted)



rag_chain = (
    {
        "context": lambda x: format_documents(x['documents']),
        "question": lambda x : x['question'],
        "chat_history": lambda _: format_chat_history(), 
        
    }
    |rag_prompt
    |llm
    |output_parser
)

web_chain = (
    {
        "context": lambda x: format_documents(x['documents']),
        "question": lambda x: x['question'],
        "chat_history": lambda _: format_chat_history(),
    }
    |web_prompt
    |llm
    |output_parser
)
# generate = rag_chain.invoke({"question":"what is negation","documents":docs})


#3.Rewrite Query
system = """You are a question rewriter, rewrite the question from the user in an optimized way without changin its semantic meaning for web search"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system),
        ("human","Here is the initial user question : {question} formualte in a better optimized way ")
    ]
)
rewriter = prompt|llm|output_parser
# print(rewriter.invoke("What is the name shake value"))

#4.Websearch tool
google_search = TavilySearchResults()
def tavily_tool(question):
    result = google_search.invoke({"query":question})
    return result

#Create Pydantic
class DocMetadata(BaseModel):
    document_name: str
    page_num: int
    heading: Optional[str]
    sub_heading: Optional[str]
    sub_sub_heading: Optional[str]
    snippet: Optional[List[str]] = []

class SerializableDoc(BaseModel):
    page_content: str
    metadata: dict
    

class Agent_type(TypedDict):
    question: str
    generation: str | None
    web_search: str
    documents: List[dict]

def serialize_docs(docs: list[LCDocument]) -> list[SerializableDoc]:
    return [
        {
            "page_content": d.page_content,
            "metadata": make_msgpack_safe(d.metadata)
        }
        for d in docs
    ]

#1.Retriever node
def retriever_node(state: Agent_type):
    print("____Retriever____")

    question = state["question"]
    history = format_chat_history()

    enriched_query = f"""
Conversation Context:
{history}

Current Question:
{question}
""".strip()

    try:
        docs = retriever.invoke(enriched_query)
        documents = serialize_docs(docs)
    except Exception as e:
        print("Retriever failed:", e)
        documents = []

    return {
        "question": question,
        "documents": documents,
        "web_search": "no",
        "generation": None
    }

# retrieved_state = retriever_node({"question":"What is negation"})
# print(retrieved_state)

#2. Generate node
def generate_node(state: Agent_type):
    print("____Generate____")

    question = state["question"]
    documents = state["documents"]

    if state["web_search"] == "yes":
        generation = web_chain.invoke(state)
    else:
        generation = rag_chain.invoke(state)

    #Save conversation
    memory.save_context(
        {"input": question},
        {"output": generation}
    )

    return {
        "question": question,
        "documents": documents,
        "generation": generation,
        "web_search": state["web_search"]
    }

#3. Grade_node
def grade_node(state: Agent_type):
    print("____Grader____")

    question = state["question"]
    documents = state.get("documents", [])

    filtered_docs = []
    web_search = "no"

    for doc in documents:
        content = doc["page_content"]

        if not content.strip():
            web_search = "yes"
            continue

        score = grader.invoke({
            "question": question,
            "documents": content,
            "chat_history": format_chat_history()
        })

        if score.binary_score == "Yes":
            filtered_docs.append(doc)
        else:
            web_search = "yes"

    if not filtered_docs:
        web_search = "yes"

    return {
        "question": question,
        "documents": filtered_docs,
        "web_search": web_search
    }


#4. Rewriter Node
def rewriter_node(state:Agent_type):
    question = state['question']
    rewrited_query = rewriter.invoke({"question":question})
    return {"question":rewrited_query,"documents": state["documents"],
        "web_search": state["web_search"]}

#5. Web search
def web_search_node(state: Agent_type):
    question = state["question"]
    documents = state["documents"]

    results = tavily_tool(question)
    web_text = "\n".join(
        r["content"] if isinstance(r, dict) else str(r)
        for r in results
    )

    documents.append({
        "page_content": web_text,
        "metadata": {"source": "web"}
    })

    return {
        "question": question,
        "documents": documents
    }


#6. Decision Maker
def decision_to_generate_node(state:Agent_type):
    question = state['question']
    documents = state['documents']
    web_search = state['web_search']

    if web_search == "yes":
        print("Question will be transformed by rewriting the query in a way that web search can able to understand")
        return "rewriter_node"
    
    else:
        return "generate_node"

workflow = StateGraph(Agent_type)
workflow.add_node("Retriever",retriever_node)
workflow.add_node("Generation",generate_node)
workflow.add_node("Web search",web_search_node)
workflow.add_node("Rewriter",rewriter_node)
workflow.add_node("Grader",grade_node)
workflow.set_entry_point("Retriever")
workflow.add_edge("Retriever","Grader")
workflow.add_conditional_edges("Grader",decision_to_generate_node,{"rewriter_node":"Rewriter","generate_node":"Generation"})
workflow.add_edge("Rewriter","Web search")
workflow.add_edge("Web search","Generation")
workflow.add_edge("Generation",END)
app = workflow.compile(checkpointer=MemorySaver())
config = {"configurable":{"thread_id":1}}

answer = app.invoke({"question":"What is the use of tavily search"},config,stream_mode="values")
print(answer['generation'])











INFO:astrapy.data.cursors.cursor:cursor fetching a page: (empty page state) from Bot_collection_HIL


____Retriever____


INFO:httpx:HTTP Request: POST https://da560a44-8a47-47c1-9200-eb2f9e31c431-us-east1.apps.astra.datastax.com/api/json/v1/default_keyspace/Bot_collection_HIL "HTTP/1.1 200 OK"
INFO:astrapy.data.cursors.cursor:cursor finished fetching a page: (empty page state) from Bot_collection_HIL


____Grader____


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Question will be transformed by rewriting the query in a way that web search can able to understand


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


____Generate____


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


It seems like the web results provided are not relevant to the question about Tavily search formulation. The content appears to be a jumbled collection of letters and symbols, possibly a fragment of an HTTP error message.

To better answer your question, I'll try to provide a general response. Tavily search formulation is not a widely recognized term, and without more context, it's challenging to provide specific applications, benefits, or advantages. However, I can suggest some possible areas where search formulation might be utilized:

1. **Information Retrieval**: Search formulation can be crucial in retrieving relevant information from large datasets or databases. A well-crafted search query can help users find the desired information efficiently.
2. **Natural Language Processing (NLP)**: Search formulation can be applied to NLP tasks, such as text classification, sentiment analysis, or question-answering systems.
3. **Web Search**: Search formulation can be used to improve web sea

In [27]:
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [ ]:
pip show langgraph

Name: langgraph
Version: 1.0.3
Summary: Building stateful, multi-actor applications with LLMs
Home-page: https://docs.langchain.com/oss/python/langgraph/overview
Author: 
Author-email: 
License-Expression: MIT
Location: c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-prebuilt, langgraph-sdk, pydantic, xxhash
Required-by: langchain
Note: you may need to restart the kernel to use updated packages.
